# FinDER KG: Full Comparison Experiment (Clean)

## 4가지 방법 비교 실험

| 실험 | 방법 | 설명 |
|------|------|------|
| **[A]** | LLM Only | Context 없이 LLM만 사용 |
| **[B]** | LLM + Text RAG | References 텍스트를 context로 사용 |
| **[C]** | LLM + Graph RAG (LPG) | Message-Passing GNN (GAT) |
| **[D]** | LLM + Graph RAG (RDF) | Knowledge Graph Embedding (TransE) |

**Requirements**: A100 GPU, High RAM runtime

**결과 저장**: Google Drive에 자동 저장됨

---
## 1. Environment Setup & Installations

In [ ]:
# GPU 확인
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

In [ ]:
# 필요한 패키지 설치
!pip install -q torch torch_geometric neo4j transformers sentence-transformers tqdm pandas
!pip install -q pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

In [ ]:
import torch
import pandas as pd
import json
import ast
import os
from datetime import datetime
from tqdm import tqdm

# Device 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EMBEDDING_DIM = 384
print(f"Device: {DEVICE}")
print(f"PyTorch: {torch.__version__}")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 2. Google Drive Mount & Result Paths

In [ ]:
from google.colab import drive

# Google Drive 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 결과 저장 디렉토리 설정
RESULT_DIR = '/content/drive/MyDrive/Colab Notebooks/kg-fibo-finder/experiment_results'
os.makedirs(RESULT_DIR, exist_ok=True)

# 실험 ID (타임스탬프)
EXPERIMENT_ID = datetime.now().strftime('%Y%m%d_%H%M%S')
print(f"Experiment ID: {EXPERIMENT_ID}")
print(f"Results will be saved to: {RESULT_DIR}")

def save_checkpoint(data, name):
    """중간 결과 저장"""
    filepath = f"{RESULT_DIR}/{EXPERIMENT_ID}_{name}.csv"
    if isinstance(data, pd.DataFrame):
        data.to_csv(filepath, index=False)
    else:
        pd.DataFrame(data).to_csv(filepath, index=False)
    print(f"✅ Saved checkpoint: {filepath}")
    return filepath

def save_log(message):
    """로그 저장"""
    log_path = f"{RESULT_DIR}/{EXPERIMENT_ID}_log.txt"
    with open(log_path, 'a') as f:
        f.write(f"[{datetime.now().strftime('%H:%M:%S')}] {message}\n")
    print(message)

In [ ]:
# FinDER KG Merged Parquet 로드
PARQUET_PATH = '/content/drive/MyDrive/Colab Notebooks/kg-fibo-finder/FinDER_KG_Merged.parquet'

if os.path.exists(PARQUET_PATH):
    df = pd.read_parquet(PARQUET_PATH)
    save_log(f"Loaded FinDER_KG_Merged.parquet: {len(df)} rows")
    print(f"Columns: {df.columns.tolist()}")
    display(df.head(3))
else:
    save_log(f"File not found: {PARQUET_PATH}, trying HuggingFace...")
    df = pd.read_parquet("hf://datasets/Linq-AI-Research/FinDER/data/train-00000-of-00001.parquet")
    save_log(f"Loaded from HuggingFace: {len(df)} rows")

---
## 3. HuggingFace Login

In [ ]:
from huggingface_hub import login

# Set your HuggingFace token (get from https://huggingface.co/settings/tokens)
HF_TOKEN = os.environ.get("HF_TOKEN", "YOUR_HF_TOKEN_HERE")
login(token=HF_TOKEN)
save_log("HuggingFace 로그인 완료!")

---
## 4. Neo4j Connection (ngrok)

⚠️ **로컬에서 Neo4j + ngrok 실행 필요**

In [ ]:
# ngrok 주소 설정 (로컬에서 ngrok tcp 7687 실행 후 나오는 주소)
NGROK_ADDRESS = "0.tcp.jp.ngrok.io:17320"  # <-- 본인 ngrok 주소로 변경!

NEO4J_URI = f"bolt://{NGROK_ADDRESS}"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

print(f"Neo4j URI: {NEO4J_URI}")

In [ ]:
from neo4j import GraphDatabase

NEO4J_CONNECTED = False

try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    driver.verify_connectivity()
    save_log("Neo4j 연결 성공!")
    NEO4J_CONNECTED = True
    
    for db_name in ["finderlpg", "finderrdf"]:
        try:
            with driver.session(database=db_name) as session:
                result = session.run("MATCH (n) RETURN count(n) as count")
                count = result.single()["count"]
                save_log(f"  {db_name}: {count:,} nodes")
        except Exception as e:
            save_log(f"  {db_name}: Error - {e}")
            
except Exception as e:
    save_log(f"Neo4j 연결 실패: {e}")
    save_log("⚠️ Graph RAG 실험 [C], [D]는 스킵됩니다. LLM Only, Text RAG만 실행합니다.")

---
## 5. Load Models (LLM + Embedder)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# Sentence Transformer
save_log("Loading SentenceTransformer...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
save_log(f"Embedder loaded: {EMBEDDING_DIM}-dim embeddings")

# LLM (Llama 3.1)
LLM_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"

save_log(f"Loading LLM: {LLM_MODEL}")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

llm = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"
)
llm.eval()
save_log(f"LLM loaded on: {next(llm.parameters()).device}")

---
## 6. GNN Models Definition

In [ ]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.nn.kge import TransE

class MessagePassingGNN(torch.nn.Module):
    """LPG용 GNN: GAT"""
    def __init__(self, input_dim, hidden_dim=256, output_dim=384, num_layers=2, heads=4, dropout=0.1):
        super().__init__()
        self.input_proj = torch.nn.Linear(input_dim, hidden_dim)
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        for i in range(num_layers):
            in_ch = hidden_dim if i == 0 else hidden_dim * heads
            self.convs.append(GATConv(in_ch, hidden_dim, heads=heads, dropout=dropout))
            self.norms.append(torch.nn.LayerNorm(hidden_dim * heads))
        self.output_proj = torch.nn.Linear(hidden_dim * heads, output_dim)
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, x, edge_index, edge_type=None):
        x = torch.relu(self.input_proj(x))
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index)
            x = norm(x)
            x = torch.relu(x)
            x = self.dropout(x)
        return self.output_proj(x.mean(dim=0))

class TransEEncoder(torch.nn.Module):
    """RDF용 KGE: TransE"""
    def __init__(self, num_nodes, num_relations, hidden_dim=256, output_dim=384):
        super().__init__()
        self.transe = TransE(num_nodes=num_nodes, num_relations=num_relations, 
                             hidden_channels=hidden_dim, margin=1.0, p_norm=1.0)
        self.output_proj = torch.nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
    
    def forward(self, head_index, rel_type, tail_index):
        if len(head_index) == 0:
            return torch.zeros(self.output_dim, device=head_index.device)
        head_emb = self.transe.node_emb(head_index)
        rel_emb = self.transe.rel_emb(rel_type)
        return self.output_proj((head_emb + rel_emb).mean(dim=0))

# 모델 초기화
save_log("Initializing GNN models...")
gnn_lpg = MessagePassingGNN(input_dim=EMBEDDING_DIM).to(DEVICE)
save_log(f"MessagePassingGNN (LPG/GAT): {sum(p.numel() for p in gnn_lpg.parameters()):,} params")

NUM_NODES_RDF, NUM_RELATIONS_RDF = 15000, 100
kge_rdf = TransEEncoder(num_nodes=NUM_NODES_RDF, num_relations=NUM_RELATIONS_RDF, 
                        hidden_dim=256, output_dim=EMBEDDING_DIM).to(DEVICE)
save_log(f"TransEEncoder (RDF/TransE): {sum(p.numel() for p in kge_rdf.parameters()):,} params")

---
## 7. Helper Functions

In [ ]:
entity_embeddings = {}

def get_entity_embedding(entity_text):
    if entity_text not in entity_embeddings:
        entity_embeddings[entity_text] = embedder.encode(entity_text, convert_to_tensor=True)
    return entity_embeddings[entity_text]

def generate_response(question, context=None, max_new_tokens=256):
    if context:
        messages = [
            {"role": "system", "content": "You are a financial expert. Answer based ONLY on the provided context. Be concise."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a financial expert. Answer concisely."},
            {"role": "user", "content": question}
        ]
    
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(llm.device)
    with torch.no_grad():
        outputs = llm.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=False, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True).strip()

def load_subgraph_lpg(driver, question_id, max_hops=2):
    query = f"""
    MATCH (e:Entity) WHERE $qid IN e.question_ids
    WITH collect(e) as seed_nodes UNWIND seed_nodes as seed
    OPTIONAL MATCH path = (seed)-[*1..{max_hops}]-(connected:Entity)
    WITH seed_nodes + collect(DISTINCT connected) as all_nodes UNWIND all_nodes as n
    WITH DISTINCT n OPTIONAL MATCH (n)-[r]->(m:Entity)
    RETURN collect(DISTINCT {{id: n.id, label: n.label, name: coalesce(n.name, n.id)}}) as nodes,
           collect(DISTINCT {{source: startNode(r).id, target: endNode(r).id, type: type(r)}}) as edges
    """
    try:
        with driver.session(database="finderlpg") as session:
            record = session.run(query, qid=question_id).single()
        if record:
            return {"nodes": record["nodes"], "edges": [e for e in record["edges"] if e["source"] and e["target"]]}
    except: pass
    return {"nodes": [], "edges": []}

def load_subgraph_rdf(driver, question_id):
    query = """MATCH (r:Resource)-[rel]-(r2:Resource) WITH r, r2, rel LIMIT 100
    RETURN collect(DISTINCT {id: r.uri, uri: r.uri}) + collect(DISTINCT {id: r2.uri, uri: r2.uri}) as nodes,
           collect(DISTINCT {source: r.uri, target: r2.uri, type: type(rel)}) as edges"""
    try:
        with driver.session(database="finderrdf") as session:
            record = session.run(query, qid=question_id).single()
        if record:
            seen = set()
            unique_nodes = [n for n in record["nodes"] if n and n.get("id") and n["id"] not in seen and not seen.add(n["id"])]
            return {"nodes": unique_nodes[:50], "edges": [e for e in record["edges"] if e["source"] and e["target"]][:100]}
    except: pass
    return {"nodes": [], "edges": []}

def build_lpg_graph(subgraph, embedding_dim):
    nodes, edges = subgraph.get("nodes", []), subgraph.get("edges", [])
    if not nodes: return None
    node_to_idx = {n["id"]: i for i, n in enumerate(nodes)}
    node_texts = [f"{n.get('label', '')}: {n.get('name', n['id'])}" for n in nodes]
    x = torch.stack([get_entity_embedding(t) for t in node_texts])
    edge_index, edge_descs = [], []
    for e in edges:
        src, tgt = e.get("source"), e.get("target")
        if src in node_to_idx and tgt in node_to_idx:
            edge_index.append([node_to_idx[src], node_to_idx[tgt]])
            edge_descs.append(f"{src} --{e.get('type', 'rel')}--> {tgt}")
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous() if edge_index else torch.zeros((2, 0), dtype=torch.long)
    data = Data(x=x, edge_index=edge_index)
    data.node_descs, data.edge_descs = node_texts, edge_descs
    return data

---
## 8. Experiment Functions

In [ ]:
NODE_TO_IDX, RELATION_TO_IDX = {}, {}

def run_llm_only(question):
    return generate_response(question)

def run_text_rag(question, references):
    try:
        parsed = ast.literal_eval(references) if isinstance(references, str) else references
        context = "\n".join(parsed) if isinstance(parsed, list) else str(references)
    except: context = str(references) if references else ""
    return generate_response(question, context=context) if context else run_llm_only(question)

def run_graph_rag_lpg(question, question_id, driver, gnn, device):
    subgraph = load_subgraph_lpg(driver, question_id)
    pyg_data = build_lpg_graph(subgraph, EMBEDDING_DIM)
    if pyg_data is None or pyg_data.x.shape[0] == 0:
        return run_llm_only(question), {"nodes": 0, "edges": 0}
    pyg_data = pyg_data.to(device)
    gnn.eval()
    with torch.no_grad(): _ = gnn(pyg_data.x, pyg_data.edge_index)
    context = "=== [LPG] Entities ===\n" + "\n".join(f"- {d}" for d in pyg_data.node_descs[:20])
    context += "\n\n=== Relationships ===\n" + "\n".join(f"- {d}" for d in pyg_data.edge_descs[:30])
    return generate_response(question, context), {"nodes": pyg_data.x.shape[0], "edges": pyg_data.edge_index.shape[1]}

def run_graph_rag_rdf(question, question_id, driver, kge_model, device):
    global NODE_TO_IDX, RELATION_TO_IDX
    subgraph = load_subgraph_rdf(driver, question_id)
    edges = subgraph.get('edges', [])
    if not edges: return run_llm_only(question), {"nodes": 0, "edges": 0, "relations": 0}
    head_indices, rel_types, tail_indices, edge_descs = [], [], [], []
    for e in edges:
        src, tgt, rel = e.get('source'), e.get('target'), e.get('type') or 'related'
        if src and tgt:
            if src not in NODE_TO_IDX: NODE_TO_IDX[src] = len(NODE_TO_IDX) % kge_model.transe.num_nodes
            if tgt not in NODE_TO_IDX: NODE_TO_IDX[tgt] = len(NODE_TO_IDX) % kge_model.transe.num_nodes
            if rel not in RELATION_TO_IDX: RELATION_TO_IDX[rel] = len(RELATION_TO_IDX) % kge_model.transe.num_relations
            head_indices.append(NODE_TO_IDX[src]); rel_types.append(RELATION_TO_IDX[rel]); tail_indices.append(NODE_TO_IDX[tgt])
            edge_descs.append(f"{src} --{rel}--> {tgt}")
    if not head_indices: return run_llm_only(question), {"nodes": 0, "edges": 0, "relations": 0}
    kge_model.eval()
    with torch.no_grad(): _ = kge_model(torch.tensor(head_indices, device=device), torch.tensor(rel_types, device=device), torch.tensor(tail_indices, device=device))
    context = "=== [RDF/TransE] Triples ===\n" + "\n".join(f"- {d}" for d in edge_descs[:40])
    return generate_response(question, context), {"nodes": len(set(head_indices + tail_indices)), "edges": len(head_indices), "relations": len(set(rel_types))}

---
## 9. Run Full Comparison (with Auto-Save)

In [ ]:
def run_full_comparison(df, sample_indices, driver, gnn_lpg, kge_rdf, device, save_every=5):
    """
    4가지 방법 비교 실험 (중간 저장 포함)
    """
    results = []
    
    for i, idx in enumerate(tqdm(sample_indices, desc="Running experiments")):
        try:
            row = df.iloc[idx]
            question_id = row['_id']
            question = row['text']
            ground_truth = row['answer']
            references = row.get('references', '')
            category = row.get('category', '')
            
            result = {
                'idx': idx, 'question_id': question_id, 'question': question,
                'ground_truth': ground_truth, 'category': category,
            }
            
            # [A] LLM Only
            result['response_llm_only'] = run_llm_only(question)
            
            # [B] Text RAG
            result['response_text_rag'] = run_text_rag(question, references)
            result['has_references'] = bool(references)
            
            # [C] & [D] Graph RAG (Neo4j 연결시에만)
            if NEO4J_CONNECTED:
                result['response_graph_lpg'], stats_lpg = run_graph_rag_lpg(question, question_id, driver, gnn_lpg, device)
                result['lpg_nodes'], result['lpg_edges'] = stats_lpg['nodes'], stats_lpg['edges']
                
                result['response_graph_rdf'], stats_rdf = run_graph_rag_rdf(question, question_id, driver, kge_rdf, device)
                result['rdf_nodes'], result['rdf_edges'] = stats_rdf['nodes'], stats_rdf['edges']
                result['rdf_relations'] = stats_rdf.get('relations', 0)
            else:
                result['response_graph_lpg'] = "[SKIPPED - Neo4j not connected]"
                result['response_graph_rdf'] = "[SKIPPED - Neo4j not connected]"
                result['lpg_nodes'] = result['lpg_edges'] = 0
                result['rdf_nodes'] = result['rdf_edges'] = result['rdf_relations'] = 0
            
            results.append(result)
            torch.cuda.empty_cache()
            
            # 중간 저장 (save_every 샘플마다)
            if (i + 1) % save_every == 0:
                save_checkpoint(results, f"checkpoint_{i+1}")
                save_log(f"Progress: {i+1}/{len(list(sample_indices))} samples completed")
                
        except Exception as e:
            save_log(f"Error at sample {idx}: {str(e)}")
            continue
    
    return pd.DataFrame(results)

In [ ]:
# ============================================
# 실험 시작: 10개 샘플 테스트
# ============================================
save_log("=" * 50)
save_log("Starting experiment: 10 samples test")
save_log("=" * 50)

results_10 = run_full_comparison(
    df, sample_indices=range(10),
    driver=driver if NEO4J_CONNECTED else None,
    gnn_lpg=gnn_lpg, kge_rdf=kge_rdf, device=DEVICE,
    save_every=5
)

save_checkpoint(results_10, "results_10_final")
save_log(f"✅ Completed 10 samples test!")
display(results_10[['question', 'response_llm_only', 'response_text_rag']].head())

---
## 10. Full Evaluation (50 samples)

In [ ]:
# ============================================
# 본격 실험: 50개 샘플
# ============================================
EVAL_SAMPLES = 50

save_log("=" * 50)
save_log(f"Starting full evaluation: {EVAL_SAMPLES} samples")
save_log("=" * 50)

results_50 = run_full_comparison(
    df, sample_indices=range(min(EVAL_SAMPLES, len(df))),
    driver=driver if NEO4J_CONNECTED else None,
    gnn_lpg=gnn_lpg, kge_rdf=kge_rdf, device=DEVICE,
    save_every=10
)

save_checkpoint(results_50, "results_50_final")
save_log(f"✅ Completed {len(results_50)} samples evaluation!")

---
## 11. Results Summary

In [ ]:
import matplotlib.pyplot as plt

print("\n" + "="*70)
print("EXPERIMENT SUMMARY")
print("="*70)

# 최종 결과 사용
final_results = results_50 if 'results_50' in dir() else results_10

print(f"\nTotal samples: {len(final_results)}")
print(f"\nCategories:")
print(final_results['category'].value_counts())

print(f"\n--- Response Lengths ---")
for method, name in [('response_llm_only', 'LLM Only'), ('response_text_rag', 'Text RAG'), 
                     ('response_graph_lpg', 'Graph LPG'), ('response_graph_rdf', 'Graph RDF')]:
    if method in final_results.columns:
        avg_len = final_results[method].str.len().mean()
        print(f"  {name}: {avg_len:.0f} chars")

if NEO4J_CONNECTED:
    print(f"\n--- Graph Statistics ---")
    print(f"  LPG - Avg nodes: {final_results['lpg_nodes'].mean():.1f}, Avg edges: {final_results['lpg_edges'].mean():.1f}")
    print(f"  RDF - Avg nodes: {final_results['rdf_nodes'].mean():.1f}, Avg edges: {final_results['rdf_edges'].mean():.1f}")

# Summary 저장
summary = {
    'total_samples': len(final_results),
    'avg_len_llm_only': final_results['response_llm_only'].str.len().mean(),
    'avg_len_text_rag': final_results['response_text_rag'].str.len().mean(),
    'neo4j_connected': NEO4J_CONNECTED,
    'experiment_id': EXPERIMENT_ID,
    'finish_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}
pd.DataFrame([summary]).to_csv(f"{RESULT_DIR}/{EXPERIMENT_ID}_summary.csv", index=False)
save_log(f"Summary saved!")

In [ ]:
# 개별 결과 비교
def display_comparison(idx):
    row = final_results.iloc[idx]
    print(f"\n{'='*80}")
    print(f"Sample {idx}: {row['question'][:100]}...")
    print(f"\nCategory: {row['category']}")
    print(f"\n[Ground Truth] {str(row['ground_truth'])[:200]}...")
    print(f"\n[A] LLM Only: {row['response_llm_only'][:200]}...")
    print(f"\n[B] Text RAG: {row['response_text_rag'][:200]}...")
    if NEO4J_CONNECTED:
        print(f"\n[C] Graph LPG ({row['lpg_nodes']} nodes): {row['response_graph_lpg'][:200]}...")
        print(f"\n[D] Graph RDF ({row['rdf_nodes']} nodes): {row['response_graph_rdf'][:200]}...")

display_comparison(0)

---
## 12. Cleanup & Final Save

In [ ]:
# 최종 정리
if NEO4J_CONNECTED:
    driver.close()
    save_log("Neo4j connection closed")

torch.cuda.empty_cache()
save_log("GPU memory cleared")

save_log(f"\n{'='*50}")
save_log(f"🎉 EXPERIMENT COMPLETED!")
save_log(f"{'='*50}")
save_log(f"Results saved to: {RESULT_DIR}")
save_log(f"Experiment ID: {EXPERIMENT_ID}")
save_log(f"Files:")
for f in os.listdir(RESULT_DIR):
    if EXPERIMENT_ID in f:
        save_log(f"  - {f}")

print(f"\n\n🌙 Good night! Check results in Google Drive tomorrow.")
print(f"Path: {RESULT_DIR}")